

```
usage: rve-backend.py [-h] [-i INPUT] [-o OUTPUT] [-l OVERLAP] [-b BACKEND] [--upscale_model UPSCALE_MODEL]
                      [--interpolate_model INTERPOLATE_MODEL] [--interpolate_factor INTERPOLATE_FACTOR]
                      [--precision PRECISION] [--tensorrt_opt_profile TENSORRT_OPT_PROFILE]
                      [--scene_detect_method SCENE_DETECT_METHOD] [--scene_detect_threshold SCENE_DETECT_THRESHOLD]
                      [--overwrite] [--border_detect] [--crf CRF]
                      [--video_encoder_preset {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc,av1_nvenc,x264_vaapi,x265_vaapi,av1_vaapi}]
                      [--video_pixel_format {yuv420p,yuv422p,yuv444p,yuv420p10le,yuv422p10le,yuv444p10le}]
                      [--audio_encoder_preset {aac,libmp3lame,opus,copy_audio}]
                      [--subtitle_encoder_preset {srt,ass,webvtt,copy_subtitle}] [--audio_bitrate AUDIO_BITRATE]
                      [--custom_encoder CUSTOM_ENCODER] [--tilesize TILESIZE] [--pytorch_gpu_id PYTORCH_GPU_ID]
                      [--ncnn_gpu_id NCNN_GPU_ID] [--benchmark] [--UHD_mode] [--slomo_mode] [--hdr_mode]
                      [--dynamic_scaled_optical_flow] [--ensemble]
                      [--preview_shared_memory_id PREVIEW_SHARED_MEMORY_ID] [--output_to_mpv] [--list_backends]
                      [--pause_shared_memory_id PAUSE_SHARED_MEMORY_ID]
                      [--upscale_output_resolution UPSCALE_OUTPUT_RESOLUTION]

Backend to RVE, used to upscale and interpolate videos

options:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        input video path
  -o OUTPUT, --output OUTPUT
                        output video path or PIPE
  -l OVERLAP, --overlap OVERLAP
                        overlap size on tiled rendering (default=10)
  -b BACKEND, --backend BACKEND
                        backend used to upscale image. (pytorch/ncnn/tensorrt/directml, default=pytorch)
  --upscale_model UPSCALE_MODEL
                        Direct path to upscaling model, will automatically upscale if model is valid.
  --interpolate_model INTERPOLATE_MODEL
                        Direct path to interpolation model, will automatically interpolate if model is valid.
                        (Downloadable Options: [rife46, rife47, rife415, rife418, rife420, rife422, rife422lite]))
  --interpolate_factor INTERPOLATE_FACTOR
                        Multiplier for interpolation, will round up to nearest integer for interpolation but the fps
                        will be correct
  --precision PRECISION
                        sets precision for model, (auto/float16/float32, default=auto)
  --tensorrt_opt_profile TENSORRT_OPT_PROFILE
                        sets tensorrt optimization profile for model, (1/2/3/4/5, default=3)
  --scene_detect_method SCENE_DETECT_METHOD
                        Scene change detection to avoid interpolating transitions. (options=mean, mean_segmented,
                        none) Mean segmented splits up an image, and if an arbitrary number of segments changes are
                        detected within the segments, it will trigger a scene change. (lower sensativity thresholds
                        are not recommended)
  --scene_detect_threshold SCENE_DETECT_THRESHOLD
                        Scene change detection sensitivity, lower number means it has a higher chance of detecting
                        scene changes, with risk of detecting too many.
  --overwrite           Overwrite output video if it already exists.
  --border_detect       Detects current borders and removes them, useful for removing black bars.
  --crf CRF             Constant rate factor for videos, lower setting means higher quality.
  --video_encoder_preset {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc,av1_nvenc,x264_vaapi,x265_vaapi,av1_vaapi}
                        encoder preset that sets default encoder settings useful for hardware encoders.
  --video_pixel_format {yuv420p,yuv422p,yuv444p,yuv420p10le,yuv422p10le,yuv444p10le}
                        pixel format for output video
  --audio_encoder_preset {aac,libmp3lame,opus,copy_audio}
                        encoder preset that sets default encoder settings
  --subtitle_encoder_preset {srt,ass,webvtt,copy_subtitle}
                        encoder preset that sets default encoder settings
  --audio_bitrate AUDIO_BITRATE
                        bitrate for audio if preset is used
  --custom_encoder CUSTOM_ENCODER
                        custom encoder
  --tilesize TILESIZE   upscale images in smaller chunks, default is the size of the input video
  --pytorch_gpu_id PYTORCH_GPU_ID
                        GPU ID for pytorch backend, default is 0
  --ncnn_gpu_id NCNN_GPU_ID
                        GPU ID for ncnn backend, default is 0
  --benchmark           Benchmark without saving video
  --UHD_mode            Lowers the resoltion flow is calculated at, speeding up model and saving vram. Helpful for
                        higher resultions.
  --slomo_mode          Instead of increasing framerate, it will remain the same while just increasing the length of
                        the video.
  --hdr_mode            Appends ffmpeg command to re encode with hdr colorspace
  --dynamic_scaled_optical_flow
                        Scale the optical flow based on the difference between frames, currently only works with the
                        pytorch backend.
  --ensemble            Use ensemble when interpolating if the model supports it.
  --preview_shared_memory_id PREVIEW_SHARED_MEMORY_ID
                        Memory ID to share preview on
  --output_to_mpv       Outputs to mpv instead of an output file (requires mpv to be installed)
  --list_backends       list out available backends
  --pause_shared_memory_id PAUSE_SHARED_MEMORY_ID
                        File to store paused state (True means paused, False means unpaused)
```




In [ ]:
# File Settings
INPUT_FILE = "input.mp4"

# this is the output folder that will me exported to the "My Drive" folder in Google Drive
OUTPUT_FILE = "output.mkv" # mkv is recommended, as it supports more codecs {mkv, mp4, avi, mov}

# Interpolate Settings

# https://github.com/TNTwise/real-video-enhancer-models/releases/tag/models/ is where all model files for RVE are stored
# you can download them from there, or try custom models from https://openmodeldb.info/
INTERPOLATE = True # {True, False}
INTERPOLATE_FACTOR = 2
SCENE_DETECT_METHOD = "pyscenedetect"
SCENE_DETECT_SENSATIVITY = "3.5" # {0 - 9.9} lower is more sensative
INTERPOLATE_MODEL_LINK = "https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.26.pkl" # {(link to model path)}


# Upscale Settings
UPSCALE = True # {True, False}
UPSCALE_MODEL_LINK = "https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/2x_ModernSpanimationV2.pth" # {(link to model path)}

# FFMpeg Settings
VIDEO_ENCODER = "libx264" # {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc}
AUDIO_ENCODER = "copy_audio" # {aac,libmp3lame,opus,copy_audio}
SUBTITLE_ENCODER = "copy_subtitle" # {srt,ass,webvtt,copy_subtitle}

# Backend Settings
BACKEND = "pytorch" # {pytorch, tensorrt} TensorRT is fastest, but takes a long time to install deps.


In [ ]:
# will move output file to drive when done for permident storage
import os
import subprocess
import shutil
from google.colab import drive
# mount drive and move output there for permadent storage

drive.mount('/content/drive')

In [ ]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer --branch 2.2.0

In [ ]:

!mkdir real-video-enhancer/models/
origdir = os.getcwd()
os.chdir("real-video-enhancer/models/")
UPSCALE_MODEL = None
INTERPOLATE_MODEL = None
def has_model(model: str):
    return model and model.lower() != "none"

def download_model(model: str):
    os.system(f"wget {model}")

# get interpolate model by extracting the end of the link
if has_model(INTERPOLATE_MODEL_LINK) and INTERPOLATE:
    download_model(INTERPOLATE_MODEL_LINK)
    INTERPOLATE_MODEL = INTERPOLATE_MODEL_LINK.split(r'/')[-1]

if has_model(UPSCALE_MODEL_LINK) and UPSCALE:
    download_model(UPSCALE_MODEL_LINK)
    UPSCALE_MODEL = UPSCALE_MODEL_LINK.split(r'/')[-1]

os.chdir(origdir)



In [ ]:
if BACKEND.lower() == 'tensorrt':
    !pip install  --pre --extra-index-url https://download.pytorch.org/whl/test/cu126 -r real-video-enhancer/backend/requirements.txt # only install if using tensorrt

!mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/ffmpeg-colab/ffmpeg && chmod +x ffmpeg

In [ ]:
#example upscale/denoise command
# It is recommended to use  --video_encoder_preset x264_nvenc
def build_backend_command() -> str:
    command = ('python3 real-video-enhancer/backend/rve-backend.py'
               + f' -i "{INPUT_FILE}"'
               + f' -o "{OUTPUT_FILE}"'
               + f' -b {BACKEND} '
               + f' --video_encoder_preset {VIDEO_ENCODER}'
               + f' --audio_encoder_preset {AUDIO_ENCODER}'
               + f' --subtitle_encoder_preset {SUBTITLE_ENCODER}')
    if UPSCALE_MODEL:
        command += f" --upscale_model real-video-enhancer/models/{UPSCALE_MODEL} "
    if INTERPOLATE_MODEL:
        command += f" --interpolate_model real-video-enhancer/models/{INTERPOLATE_MODEL} "
        command += f" --interpolate_factor {INTERPOLATE_FACTOR} "
        command += f" --scene_detect_method {SCENE_DETECT_METHOD} "
        command += f" --scene_detect_threshold {SCENE_DETECT_SENSATIVITY} "

    return command

with open("command.sh", "w") as f:
  f.write(build_backend_command())

!bash command.sh

In [ ]:
# Source file path in Colab
source_path = f'/content/{OUTPUT_FILE}'

# Destination path on Google Drive (root directory)
destination_path = f'/content/drive/My Drive/{OUTPUT_FILE}'

# Move the file
shutil.move(source_path, destination_path)